In [1]:
import random
import rospy
import time
from turtlesim.srv import Spawn
from turtlesim.srv import Kill
from turtlesim.srv import SetPen
from turtlesim.srv import TeleportAbsolute
from geometry_msgs.msg import Twist
from std_msgs.msg import String
rospy.init_node("serwis_node_Leonardo")
rozkazy=[]
global rozkazy
def ustaw_pol(data):
 
    rozkazy.append(data.data)

        
class Robot:
    
     
    def __init__(self, name, x, y, theta,r,g,b):
        # inicjalizacja klasy
        self.name = name
        self.x = x
        self.y = y
        self.r = r
        self.g = g
        self.b = b
        self.theta=theta
        self.dok_x=x
        self.dok_y=y
        self.battery=0.99
        self.dok_theta=theta
        self.akt_pol="none"
        self.akt_zad="none"
        create = rospy.ServiceProxy('spawn', Spawn)
        create(self.dok_x, self.dok_y, self.dok_theta, name)
        self.pol=rospy.Publisher(name+"/polecenie",String,queue_size=10)
        self.xp=rospy.Publisher(name+"/pol_x",String,queue_size=10)
        self.yp=rospy.Publisher(name+"/pol_y",String,queue_size=10)
        self.zad=rospy.Publisher(name+"/pol_zad",String,queue_size=10)
        self.tryp_p=rospy.Publisher(name+"/tryb",String,queue_size=10)
        self.tryp_p.publish("auto")
        self.manual=rospy.Subscriber("manual",String, self.manual) 
        pen = rospy.ServiceProxy(name+"/set_pen", SetPen)
        pen(r, g, b, 5,0)
        self.tryb="auto"
        my_sube_pol=rospy.Subscriber(self.name+"/polecenie",String, ustaw_pol)
        my_sube_pol.unregister()
        my_sube_pol=rospy.Subscriber(self.name+"/polecenie",String, ustaw_pol)
        tryb=rospy.Subscriber(self.name+"/tryb",String, self.ustaw_tryb)
        tryb.unregister()
        tryb=rospy.Subscriber(self.name+"/tryb",String, self.ustaw_tryb)
        self.rysuj_dok()
        
        
    def sepuku(self):
        kill = rospy.ServiceProxy('kill', Kill)
        kill(self.name)
        
    def rezurektrion(self):
        create = rospy.ServiceProxy('spawn', Spawn)
        create(self.dok_x, self.dok_y, self.dok_theta, self.name)
        
    def pose(self):
        print("{:f}, {:f}, {:f}".format(self.x, self.y, self.theta))
    
    def pen_on(self):
            pen = rospy.ServiceProxy(self.name+'/set_pen', SetPen)
            pen(self.r, self.g, self.b, 5,0)
    def pen_off(self):
            pen = rospy.ServiceProxy(self.name+'/set_pen', SetPen)
            pen(self.r, self.g, self.b, 5,1)
    
    def obr(self, kier):
        if kier=='p':
            if self.theta==4.71:
                theta=1.57
            else:
                theta=0-self.theta
            self.theta=0
        elif kier=='g':
            theta=1.57-self.theta
            self.theta=1.57
        elif kier=='l':
            theta=3.14-self.theta
            self.theta=3.14
        elif kier=='d':
            if self.theta==0:
                theta=-1.57
            else:
                theta=4.71-self.theta
            self.theta=4.71
        pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
        msg = Twist()
        msg.linear.x = 0
        msg.angular.z = theta
        pub_speed.publish(msg)
        self.rozladuj()
        rospy.sleep(1)
    
        
    def move(self, odl):
        if self.theta==0:
            self.x+=odl
        elif self.theta==1.57:
            self.y+=odl
        elif self.theta==3.14:
            self.x-=odl
        elif self.theta==4.71:
            self.y-=odl
        pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
        msg = Twist()
        msg.linear.x = odl
        pub_speed.publish(msg)
        self.rozladuj()
        rospy.sleep(1)
      
    def go(self,x,y):
        if(self.x!=x or self.y!=y):
            if self.y==10.5:
                self.obr('d')
                self.move(1.5)
            if self.x<5.5:
                self.obr('p')
            elif self.x>5.5:
                self.obr('l')
            while self.x!=5.5:
                self.move(0.5)
                #yself.pose()
            if self.y<y:
                self.obr('g')
            elif self.y>y:
                self.obr('d')
            while self.y!=y:
                self.move(0.5)
                self.pose()
                if y>9 and self.y==9:
                    break
            if self.x<x:
                self.obr('p')
            elif self.x>x:
                self.obr('l')
            while self.x!=x:
                self.move(0.5)
                self.pose()
            if y>9:
                self.obr('g')
                while self.y!=y:
                    self.move(0.5)
    def pracuj(self):
        self.rozladuj()
        self.rozladuj()
        self.rozladuj()
        if self.theta==0:
            self.obr('g')
            self.obr('d')
            self.obr('p')
        if self.theta==3.14:
            self.obr('d')
            self.obr('g')
            self.obr('l')
        self.akt_zad="none"
        self.akt_pol="none"
        self.pow_dok()
    def laduj(self):
        if self.battery<1:
            self.battery+= random.randint(0,9)/1000
                           
    def rozladuj(self):
        if self.battery<1:
            self.battery-= random.randint(0,9)/1000
                           
    def stan_baterii(self):
            print("Bateria: {:.0f}%".format(self.battery*100))
      
    def sprawdz(self):
        
        
        if self.tryb!="manual":
            if len(rozkazy)!=0:
                    rozkaz=rozkazy[0].split("|")
                    rozkazy.pop(0)
                    x=float(rozkaz[1])
                    y=float(rozkaz[2])
                    self.akt_zad=rozkaz[0]
                    self.akt_pol=rozkaz[3]
                    self.go(x,y)     
                    if self.akt_pol!="none":
                        self.pracuj()   
            else:
                if self.dok_x!=self.x or self.dok_y!=self.y:
                    self.go(self.dok_x,self.dok_y) 
                    self.obr("d")

                else:
                    dok = rospy.ServiceProxy(self.name+'/teleport_absolute', TeleportAbsolute)
                    dok(self.dok_x, self.dok_y, 4.71)
                    while self.battery<1:
                        self.laduj();
                        if len(rozkazy)!=0:
                            break  
    def manual(self,data):
            p=data.data
            if p=="p":
                    self.move(0.5)
            elif p=="t":
                self.move(-0.5)
            elif p=="l":
                self.theta+=1.57
                pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
                msg = Twist()
                msg.linear.x = 0
                msg.angular.z = 1.57
                pub_speed.publish(msg)
            elif p=="pp":
                self.theta-=1.57
                pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
                msg = Twist()
                msg.linear.x = 0
                msg.angular.z = -1.57
                pub_speed.publish(msg)
            self.rozladuj()
            rospy.sleep(1)
            
        
      
        
    def ustaw_tryb(self,data):
        self.tryb=data.data
        
    def rysuj_dok(self):
            dok = rospy.ServiceProxy(self.name+'/teleport_absolute', TeleportAbsolute)
            self.pen_off()
            dok(self.dok_x-0.5, self.dok_y-0.5, 0)
            self.pen_on()
            dok(self.dok_x+0.5, self.dok_y-0.5, 0)
            dok(self.dok_x+0.5, self.dok_y+0.5, 0)
            dok(self.dok_x-0.5, self.dok_y+0.5, 0)
            dok(self.dok_x-0.5, self.dok_y-0.5, 0)
            self.pen_off()
            dok(self.dok_x, self.dok_y, 4.71)
            #self.pen_on()



shutdown request: [/serwis_node_Leonardo] Reason: new node registered with same name


In [3]:
Raphael=Robot("Raphael",9,10.5,4.71, 255,0,0)




ServiceException: service [/spawn] responded with an error: b''

In [3]:
print(rozkazy)

[]


In [22]:
while 1:
    
    Leonardo.sprawdz()
    time.sleep(0.5)

[ERROR] [1624567616.217644]: bad callback: <bound method Robot.ustaw_tryb of <__main__.Robot object at 0x7fc37025c7c0>>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-17-48699e4dcb8e>", line 235, in ustaw_tryb
    self.manual.unregister()
AttributeError: 'function' object has no attribute 'unregister'



5.500000, 8.500000, 4.710000
5.500000, 8.000000, 4.710000
5.500000, 7.500000, 4.710000
5.500000, 7.000000, 4.710000
5.000000, 7.000000, 3.140000
4.500000, 7.000000, 3.140000
4.000000, 7.000000, 3.140000
3.500000, 7.000000, 3.140000
3.000000, 7.000000, 3.140000
5.500000, 7.500000, 1.570000
5.500000, 8.000000, 1.570000
5.500000, 8.500000, 1.570000
5.500000, 9.000000, 1.570000
5.000000, 9.000000, 3.140000
4.500000, 9.000000, 3.140000
4.000000, 9.000000, 3.140000


[ERROR] [1624567670.192326]: bad callback: <bound method Robot.ustaw_tryb of <__main__.Robot object at 0x7fc37025c7c0>>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-17-48699e4dcb8e>", line 233, in ustaw_tryb
    self.manual=Subscriber("manual",String, self.manual)
NameError: name 'Subscriber' is not defined



KeyboardInterrupt: 

In [ ]:
Leonardo.subs_on()

In [21]:
Leonardo.tryb

'manual'

In [ ]:
Leonardo.sepuku()

In [ ]:
Leonardo.rezurektrion()

In [20]:

Leonardo.go(8,8)
Leonardo.pracuj()


KeyboardInterrupt: 

In [ ]:
Leonardo.obr('l')
Leonardo.move(3)

In [ ]:
Leonardo.obr('g')

In [ ]:
Leonardo.obr('p')

In [ ]:
Leonardo.obr('d')

In [ ]:
!rosservice call reset